# Download the SQL Dataset

In [1]:
from datasets import load_dataset

/Users/vrabinin/Documents/Github/llm-zoomcamp-capstone/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
dataset = load_dataset('shiroyasha13/llama_text_to_sql_dataset')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 184503
    })
})


In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 184503
    })
})


In [6]:
train_dataset = dataset['train']
train_dataset.save_to_disk('../data/llama_text_to_sql_dataset')
print(train_dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 184503
})


# Get Sample Database

In [18]:
!wget --directory-prefix=../data https://www.postgresqltutorial.com/wp-content/uploads/2019/05/dvdrental.zip

3069.36s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


--2024-09-15 11:57:47--  https://www.postgresqltutorial.com/wp-content/uploads/2019/05/dvdrental.zip
Herleiden van www.postgresqltutorial.com (www.postgresqltutorial.com)... 104.21.2.174, 172.67.129.129
Verbinding maken met www.postgresqltutorial.com (www.postgresqltutorial.com)|104.21.2.174|:443... verbonden.
HTTP-verzoek is verzonden; wachten op antwoord... 200 OK
Lengte: 550906 (538K) [application/zip]
Wordt opgeslagen als: ‘../data/dvdrental.zip’

dvdrental.zip       100%[===================>] 537,99K  --.-KB/s    in 0,05s   

2024-09-15 11:57:47 (10,7 MB/s) - '‘../data/dvdrental.zip’' opgeslagen [550906/550906]



In [19]:
import zipfile
import os

# Path to the zip file
zip_file_path = '../data/dvdrental.zip'

# Directory to extract the contents
extract_dir = '../data/dvdrental_db'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f'Contents extracted to {extract_dir}')

Contents extracted to ../data/dvdrental_db


In [20]:
# Check if the file exists and delete it
if os.path.exists(zip_file_path):
    os.remove(zip_file_path)
    print(f'{zip_file_path} has been deleted.')
else:
    print(f'{zip_file_path} does not exist.')

../data/dvdrental.zip has been deleted.


# Curate the Dataset

In [25]:
from datasets import load_from_disk, Dataset
import pandas as pd
import re
src_dataset = load_from_disk('../data/llama_text_to_sql_dataset')
df_dataset = src_dataset.to_pandas()
df_dataset.head()

,input,output
0,[INST] Here is a database schema: table schema...,"SELECT COUNT(*) FROM table_204_780 WHERE ""posi..."
1,[INST] Here is a database schema: table schema...,SELECT rowers FROM table_name_80 WHERE country...
2,[INST] Here is a database schema: table schema...,SELECT MAX(attendance) FROM table_name_94 WHER...
3,[INST] Here is a database schema: table schema...,SELECT years FROM table_name_83 WHERE national...
4,[INST] Here is a database schema: table schema...,"SELECT name, COUNT(name) FROM course AS T1 JOI..."


## Function to extract Instruction and Schema


In [16]:
# Function to extract schema and instruction
def extract_schema_and_instruction(text):
    #schema_pattern = r"table schema: (.*?)(?=Please write me a syntactically correct SQL statement)"
    schema_pattern = r"table schema: (.*?)(?=\n)"
    instruction_pattern = r"Please write me a syntactically correct SQL statement that answers the following question: (.*?)(?=\[\/INST\])"
    
    schema_match = re.search(schema_pattern, text)
    instruction_match = re.search(instruction_pattern, text)
    
    schema = schema_match.group(1).strip() if schema_match else None
    instruction = instruction_match.group(1).strip() if instruction_match else None
    
    return schema, instruction

In [28]:
# Apply the function to the DataFrame column
df_dataset1 = df_dataset.copy()
df_dataset1[['schema', 'instruction']] = df_dataset1['input'].apply(lambda x: pd.Series(extract_schema_and_instruction(x)))
df_dataset1[['keyword']] = 'DUMMY'

# Print the DataFrame with the new columns
df_dataset1.head()

,input,output,schema,instruction,keyword
0,[INST] Here is a database schema: table schema...,"SELECT COUNT(*) FROM table_204_780 WHERE ""posi...","table_204_780: id [ number] ""year"" [ number]...",tell me the number of times he placed 4th .It ...,DUMMY
1,[INST] Here is a database schema: table schema...,SELECT rowers FROM table_name_80 WHERE country...,table_name_80: rowers [ VARCHAR] country [ V...,Who is the rower from Canada?It is not neccess...,DUMMY
2,[INST] Here is a database schema: table schema...,SELECT MAX(attendance) FROM table_name_94 WHER...,table_name_94: attendance [ INTEGER] result ...,What is the highest attendance at a game with ...,DUMMY
3,[INST] Here is a database schema: table schema...,SELECT years FROM table_name_83 WHERE national...,table_name_83: years [ VARCHAR] nationality ...,"What is Years, when Nationality is United Stat...",DUMMY
4,[INST] Here is a database schema: table schema...,"SELECT name, COUNT(name) FROM course AS T1 JOI...",advisor: s_ID [ varchar(5)] i_ID [ varchar(5)],List the number of courses in year 2008 for ea...,DUMMY


In [34]:
# Remove records with empty schema and instruction columns
df_dataset2 = df_dataset1.dropna(subset=['schema', 'instruction'])
df_dataset2.head()

,input,output,schema,instruction,keyword
0,[INST] Here is a database schema: table schema...,"SELECT COUNT(*) FROM table_204_780 WHERE ""posi...","table_204_780: id [ number] ""year"" [ number]...",tell me the number of times he placed 4th .It ...,DUMMY
1,[INST] Here is a database schema: table schema...,SELECT rowers FROM table_name_80 WHERE country...,table_name_80: rowers [ VARCHAR] country [ V...,Who is the rower from Canada?It is not neccess...,DUMMY
2,[INST] Here is a database schema: table schema...,SELECT MAX(attendance) FROM table_name_94 WHER...,table_name_94: attendance [ INTEGER] result ...,What is the highest attendance at a game with ...,DUMMY
3,[INST] Here is a database schema: table schema...,SELECT years FROM table_name_83 WHERE national...,table_name_83: years [ VARCHAR] nationality ...,"What is Years, when Nationality is United Stat...",DUMMY
4,[INST] Here is a database schema: table schema...,"SELECT name, COUNT(name) FROM course AS T1 JOI...",advisor: s_ID [ varchar(5)] i_ID [ varchar(5)],List the number of courses in year 2008 for ea...,DUMMY


In [35]:
df_curated = df_dataset2[['keyword', 'instruction', 'schema', 'output']]
df_curated.head()

,keyword,instruction,schema,output
0,DUMMY,tell me the number of times he placed 4th .It ...,"table_204_780: id [ number] ""year"" [ number]...","SELECT COUNT(*) FROM table_204_780 WHERE ""posi..."
1,DUMMY,Who is the rower from Canada?It is not neccess...,table_name_80: rowers [ VARCHAR] country [ V...,SELECT rowers FROM table_name_80 WHERE country...
2,DUMMY,What is the highest attendance at a game with ...,table_name_94: attendance [ INTEGER] result ...,SELECT MAX(attendance) FROM table_name_94 WHER...
3,DUMMY,"What is Years, when Nationality is United Stat...",table_name_83: years [ VARCHAR] nationality ...,SELECT years FROM table_name_83 WHERE national...
4,DUMMY,List the number of courses in year 2008 for ea...,advisor: s_ID [ varchar(5)] i_ID [ varchar(5)],"SELECT name, COUNT(name) FROM course AS T1 JOI..."


In [36]:
# Create Dataset object from df_curated dataframe
curated_dataset = Dataset.from_pandas(df_curated)
print(curated_dataset)

Dataset({
    features: ['keyword', 'instruction', 'schema', 'output', '__index_level_0__'],
    num_rows: 184206
})


In [37]:
curated_dataset.save_to_disk('../data/rag_dataset')

Saving the dataset (1/1 shards): 100%|██████████| 184206/184206 [00:00<00:00, 1844084.21 examples/s]


# Extract Schema from DVDRENTAL Database

In [7]:
import os, psycopg2, pandas as pd
# Database connection parameters
db_params = {
    'dbname': 'dvdrental',
    'user': os.environ['POSTGRES_USER'],
    'password': os.environ['POSTGRES_PASSWORD'],
    'host': 'localhost',
    'port': os.environ['POSTGRES_PORT']
}

In [11]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

In [12]:
# SQL query to retrieve schema information
schema_query = """
SELECT 
    c.table_name, 
    CASE 
        WHEN t.table_type = 'BASE TABLE' THEN 'table'
        WHEN t.table_type = 'VIEW' THEN 'view'
    END AS table_type,    
    c.column_name, 
    c.data_type, 
    CASE 
        WHEN c.column_name IN (
            SELECT kcu.column_name
            FROM information_schema.table_constraints tco
            JOIN information_schema.key_column_usage kcu 
                ON kcu.constraint_name = tco.constraint_name
            WHERE tco.constraint_type = 'PRIMARY KEY' AND kcu.table_name = c.table_name
        ) THEN TRUE ELSE FALSE 
    END AS is_primary_key,
    c.is_nullable
FROM information_schema.columns c
JOIN information_schema.tables t
    ON c.table_name = t.table_name
ORDER BY c.table_name, c.column_name;
"""

# Execute the query
cursor.execute(schema_query)
schema_data = cursor.fetchall()

In [13]:
# Define the DataFrame structure
columns = ['table_name', 'table_type', 'column_name', 'column_data_type', 'is_primary_key', 'is_nullable']
df = pd.DataFrame(schema_data, columns=columns)

df.head()

,table_name,table_type,column_name,column_data_type,is_primary_key,is_nullable
0,_pg_foreign_data_wrappers,view,authorization_identifier,name,False,YES
1,_pg_foreign_data_wrappers,view,fdwoptions,ARRAY,False,YES
2,_pg_foreign_data_wrappers,view,fdwowner,oid,False,YES
3,_pg_foreign_data_wrappers,view,foreign_data_wrapper_catalog,name,False,YES
4,_pg_foreign_data_wrappers,view,foreign_data_wrapper_language,character varying,False,YES


In [14]:
# Close the cursor and connection
cursor.close()
conn.close()